# Chapter 5: LangGraph Applications 실습

이 노트북은 LangGraph를 사용한 실제 애플리케이션 구축을 실습합니다.

## 환경 설정

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = input("OpenAI API Key를 입력하세요: ")

## 1. 대화형 챗봇

In [5]:
from typing import TypedDict, Annotated, Sequence, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
import operator

# 향상된 State 정의
class ChatbotState(MessagesState):
    user_info: dict
    conversation_style: str
    topic_history: list

# 노드 함수들
def analyze_intent(state: ChatbotState):
    """사용자 의도 분석"""
    messages = state["messages"]
    if not messages:
        return state
    
    last_message = messages[-1].content.lower()
    
    # 간단한 의도 분석
    if "안녕" in last_message or "hello" in last_message:
        state["conversation_style"] = "greeting"
    elif "?" in last_message:
        state["conversation_style"] = "question"
    elif "감사" in last_message or "고마" in last_message:
        state["conversation_style"] = "thanks"
    else:
        state["conversation_style"] = "general"
    
    return state

def extract_entities(state: ChatbotState):
    """엔티티 추출"""
    messages = state["messages"]
    if not messages:
        return state
    
    last_message = messages[-1].content
    
    # 이름 추출 (간단한 예제)
    if "제 이름은" in last_message or "저는" in last_message:
        import re
        pattern = r"(?:제 이름은|저는)\s+(\S+)(?:입니다|이에요|예요)?"
        match = re.search(pattern, last_message)
        if match:
            if "user_info" not in state:
                state["user_info"] = {}
            state["user_info"]["name"] = match.group(1)
    
    # 주제 추출
    topics = []
    keywords = ["파이썬", "프로그래밍", "AI", "머신러닝", "웹개발"]
    for keyword in keywords:
        if keyword.lower() in last_message.lower():
            topics.append(keyword)
    
    if topics:
        if "topic_history" not in state:
            state["topic_history"] = []
        state["topic_history"].extend(topics)
    
    return state

def generate_response(state: ChatbotState):
    """응답 생성"""
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
    
    # 시스템 프롬프트 구성
    system_content = "당신은 친절하고 도움이 되는 AI 어시스턴트입니다."
    
    # 사용자 정보 반영
    if state.get("user_info", {}).get("name"):
        system_content += f" 사용자의 이름은 {state['user_info']['name']}입니다."
    
    # 대화 스타일 반영
    style_prompts = {
        "greeting": " 친근하게 인사하세요.",
        "question": " 질문에 명확하고 도움이 되는 답변을 제공하세요.",
        "thanks": " 감사 인사에 겸손하게 응답하세요.",
        "general": " 대화를 자연스럽게 이어가세요."
    }
    system_content += style_prompts.get(state.get("conversation_style", "general"), "")
    
    # 주제 기록
    if state.get("topic_history"):
        recent_topics = list(set(state["topic_history"][-5:]))
        system_content += f" 최근 대화 주제: {', '.join(recent_topics)}"
    
    messages = [SystemMessage(content=system_content)] + state["messages"]
    response = llm.invoke(messages)
    
    return {"messages": [response]}

# 그래프 구성
def create_chatbot():
    workflow = StateGraph(ChatbotState)
    
    # 노드 추가
    workflow.add_node("analyze", analyze_intent)
    workflow.add_node("extract", extract_entities)
    workflow.add_node("respond", generate_response)
    
    # 엣지 추가
    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", "extract")
    workflow.add_edge("extract", "respond")
    workflow.add_edge("respond", END)
    
    # 메모리와 함께 컴파일
    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

# 챗봇 실행
chatbot = create_chatbot()
config = {"configurable": {"thread_id": "session1"}}

print("챗봇이 시작되었습니다. 대화를 시작하세요!\n")

# 대화 시뮬레이션
test_messages = [
    "안녕하세요!",
    "제 이름은 홍길동입니다.",
    "파이썬 프로그래밍에 대해 알고 싶어요.",
    "머신러닝도 관심있어요.",
    "제 이름을 기억하시나요?"
]

for msg in test_messages:
    print(f"사용자: {msg}")
    result = chatbot.invoke(
        {"messages": [HumanMessage(content=msg)]},
        config
    )
    print(f"챗봇: {result['messages'][-1].content}\n")

챗봇이 시작되었습니다. 대화를 시작하세요!

사용자: 안녕하세요!
챗봇: 안녕하세요! 어떻게 도와드릴까요? 😊

사용자: 제 이름은 홍길동입니다.
챗봇: 반갑습니다, 홍길동님! 어떤 이야기를 나눠볼까요? 궁금한 점이나 필요하신 정보가 있다면 말씀해 주세요.

사용자: 파이썬 프로그래밍에 대해 알고 싶어요.
챗봇: 좋아요, 파이썬은 매우 인기 있는 프로그래밍 언어로, 배우기 쉽고 다양한 용도로 사용됩니다. 어떤 부분에 대해 알고 싶으신가요? 기본 문법, 데이터 구조, 라이브러리 사용법, 아니면 특정 프로젝트에 대한 도움을 원하시나요?

사용자: 머신러닝도 관심있어요.
챗봇: 머신러닝은 정말 흥미로운 분야죠! 파이썬은 머신러닝을 구현하는 데 매우 유용한 언어로, 다양한 라이브러리와 프레임워크가 있습니다. 예를 들어, **Scikit-learn**은 기본적인 머신러닝 모델을 쉽게 구현할 수 있게 도와주고, **TensorFlow**나 **PyTorch**는 딥러닝 모델을 개발하는 데 많이 사용됩니다.

머신러닝의 기본 개념에 대해 설명해드리거나, 특정 라이브러리에 대해 더 알고 싶으신가요? 어떤 방향으로 정보를 드리면 좋을까요?

사용자: 제 이름을 기억하시나요?
챗봇: 네, 홍길동님! 항상 기억하고 있습니다. 더 궁금한 점이 있으시면 언제든지 말씀해 주세요!



## 2. SQL Query Generator

In [8]:
import sqlite3
import pandas as pd
from pydantic import BaseModel, Field
from typing import Optional, TypedDict
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END

# SQL 생성기 State
class SQLState(TypedDict):
    question: str
    schema: str
    sql_query: str
    result: str
    error: Optional[str]
    explanation: str

# 데이터베이스 설정
def setup_database():
    conn = sqlite3.connect(':memory:')
    cursor = conn.cursor()
    
    # 테이블 생성
    cursor.execute('''
        CREATE TABLE employees (
            id INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            salary REAL,
            hire_date DATE
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE projects (
            id INTEGER PRIMARY KEY,
            name TEXT,
            department TEXT,
            budget REAL,
            status TEXT
        )
    ''')
    
    # 샘플 데이터
    employees = [
        (1, '김철수', 'Engineering', 70000, '2020-01-15'),
        (2, '이영희', 'Marketing', 65000, '2019-03-20'),
        (3, '박민수', 'Engineering', 80000, '2018-07-10'),
        (4, '정수진', 'HR', 60000, '2021-02-01'),
        (5, '홍길동', 'Marketing', 72000, '2019-11-15')
    ]
    
    projects = [
        (1, 'Website Redesign', 'Marketing', 50000, 'Active'),
        (2, 'Mobile App', 'Engineering', 150000, 'Active'),
        (3, 'Data Pipeline', 'Engineering', 80000, 'Completed'),
        (4, 'Training Program', 'HR', 30000, 'Planning')
    ]
    
    cursor.executemany('INSERT INTO employees VALUES (?, ?, ?, ?, ?)', employees)
    cursor.executemany('INSERT INTO projects VALUES (?, ?, ?, ?, ?)', projects)
    conn.commit()
    
    return conn

# 전역 데이터베이스 연결
GLOBAL_CONN = None

def get_connection():
    """전역 데이터베이스 연결 관리"""
    global GLOBAL_CONN
    if GLOBAL_CONN is None:
        GLOBAL_CONN = setup_database()
    return GLOBAL_CONN

# SQL 생성 노드
def generate_sql(state: SQLState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    prompt = f"""
    다음 스키마를 가진 데이터베이스가 있습니다:
    
    {state['schema']}
    
    사용자 질문: {state['question']}
    
    위 질문에 답하기 위한 SQL 쿼리를 작성하세요.
    쿼리만 반환하고 다른 설명은 포함하지 마세요.
    """
    
    response = llm.invoke(prompt)
    sql_query = response.content.strip()
    sql_query = sql_query.replace("```sql", "").replace("```", "").strip()
    
    state["sql_query"] = sql_query
    return state

# SQL 실행 노드 (수정됨)
def execute_sql(state: SQLState):
    """SQL 실행 노드 - 전역 연결 사용"""
    conn = get_connection()  # 전역 연결 사용
    
    try:
        result = pd.read_sql_query(state["sql_query"], conn)
        state["result"] = result.to_string()
        state["error"] = None
    except Exception as e:
        state["error"] = str(e)
        state["result"] = ""
    
    return state

# 결과 설명 노드
def explain_result(state: SQLState):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    if state.get("error"):
        state["explanation"] = f"쿼리 실행 중 오류가 발생했습니다: {state['error']}"
    else:
        prompt = f"""
        사용자 질문: {state['question']}
        실행된 SQL: {state['sql_query']}
        결과:
        {state['result']}
        
        위 결과를 사용자가 이해하기 쉽게 설명해주세요.
        """
        
        response = llm.invoke(prompt)
        state["explanation"] = response.content
    
    return state

# SQL 생성기 그래프
def create_sql_generator():
    workflow = StateGraph(SQLState)
    
    # 노드 추가
    workflow.add_node("generate", generate_sql)
    workflow.add_node("execute", execute_sql)
    workflow.add_node("explain", explain_result)
    
    # 엣지 추가
    workflow.set_entry_point("generate")
    workflow.add_edge("generate", "execute")
    workflow.add_edge("execute", "explain")
    workflow.add_edge("explain", END)
    
    return workflow.compile()

# 실행
sql_generator = create_sql_generator()

# 스키마 정보
schema = """
employees 테이블:
- id: INTEGER (PRIMARY KEY)
- name: TEXT
- department: TEXT
- salary: REAL
- hire_date: DATE

projects 테이블:
- id: INTEGER (PRIMARY KEY)
- name: TEXT
- department: TEXT
- budget: REAL
- status: TEXT
"""

# 테스트 질문들
questions = [
    "Engineering 부서의 평균 급여는?",
    "가장 높은 급여를 받는 직원은?",
    "Active 상태인 프로젝트들의 총 예산은?",
    "부서별 직원 수는?"
]

for question in questions:
    print(f"\n질문: {question}")
    print("="*60)
    
    # connection을 state에 포함시키지 않고, 전역 연결 사용
    result = sql_generator.invoke({
        "question": question,
        "schema": schema
    })
    
    print(f"SQL: {result['sql_query']}")
    print(f"\n결과:\n{result['result']}")
    print(f"\n설명: {result['explanation']}")



질문: Engineering 부서의 평균 급여는?
SQL: SELECT AVG(salary) AS average_salary
FROM employees
WHERE department = 'Engineering';

결과:
   average_salary
0         75000.0

설명: 사용자가 요청한 Engineering 부서의 평균 급여는 75,000달러입니다. 즉, Engineering 부서에 근무하는 직원들의 급여를 모두 합산한 후, 직원 수로 나눈 결과가 75,000달러라는 의미입니다. 이는 해당 부서의 직원들이 평균적으로 이 정도의 급여를 받고 있다는 것을 나타냅니다.

질문: 가장 높은 급여를 받는 직원은?
SQL: SELECT * FROM employees WHERE salary = (SELECT MAX(salary) FROM employees);

결과:
   id name   department   salary   hire_date
0   3  박민수  Engineering  80000.0  2018-07-10

설명: 사용자가 요청한 "가장 높은 급여를 받는 직원"에 대한 SQL 쿼리를 실행한 결과, 박민수라는 직원이 가장 높은 급여를 받고 있음을 확인했습니다. 

그의 급여는 80,000달러이며, 소속 부서는 엔지니어링입니다. 박민수는 2018년 7월 10일에 회사에 입사했습니다. 

즉, 현재 회사에서 가장 높은 급여를 받는 직원은 박민수이며, 그의 급여는 80,000달러입니다.

질문: Active 상태인 프로젝트들의 총 예산은?
SQL: SELECT SUM(budget) AS total_budget
FROM projects
WHERE status = 'Active';

결과:
   total_budget
0      200000.0

설명: 사용자가 요청한 "Active 상태인 프로젝트들의 총 예산"에 대한 SQL 쿼리를 실행한 결과, 현재 활성 상태인 프로젝트들의 예산을 모두 합산한 결과가 200,000.0으로 나타났습니다

## 3. Multi-Source RAG System

In [9]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
from enum import Enum

class SourceType(str, Enum):
    WEB = "web"
    DATABASE = "database"
    DOCUMENT = "document"
    API = "api"

class MultiRAGState(TypedDict):
    query: str
    source_type: SourceType
    retrieved_docs: list
    answer: str
    sources_used: list

# 다양한 소스 시뮬레이션
def create_multi_source_data():
    """다양한 소스의 데이터 생성"""
    
    # 웹 데이터
    web_docs = [
        Document(
            page_content="Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.",
            metadata={"source": "web", "url": "python.org"}
        ),
        Document(
            page_content="Django는 Python으로 작성된 오픈소스 웹 프레임워크입니다.",
            metadata={"source": "web", "url": "djangoproject.com"}
        ),
    ]
    
    # 데이터베이스 데이터
    db_docs = [
        Document(
            page_content="2024년 기준 Python 개발자 평균 연봉은 8,500만원입니다.",
            metadata={"source": "database", "table": "salary_stats"}
        ),
        Document(
            page_content="가장 인기있는 Python 라이브러리는 NumPy, Pandas, TensorFlow입니다.",
            metadata={"source": "database", "table": "library_usage"}
        ),
    ]
    
    # 문서 데이터
    doc_docs = [
        Document(
            page_content="머신러닝 프로젝트를 시작할 때는 데이터 전처리가 가장 중요합니다.",
            metadata={"source": "document", "file": "ml_guide.pdf"}
        ),
        Document(
            page_content="딥러닝 모델 학습 시 과적합을 방지하기 위해 드롭아웃을 사용합니다.",
            metadata={"source": "document", "file": "dl_best_practices.pdf"}
        ),
    ]
    
    # API 데이터
    api_docs = [
        Document(
            page_content="OpenAI GPT-4의 컨텍스트 윈도우는 128,000 토큰입니다.",
            metadata={"source": "api", "endpoint": "/models/gpt-4"}
        ),
        Document(
            page_content="LangChain은 LLM 애플리케이션 개발을 위한 프레임워크입니다.",
            metadata={"source": "api", "endpoint": "/frameworks/langchain"}
        ),
    ]
    
    return web_docs + db_docs + doc_docs + api_docs

# 소스 라우터
def route_query(state: MultiRAGState):
    """쿼리를 분석하여 적절한 소스 결정"""
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    class SourceDecision(BaseModel):
        source_types: list[SourceType] = Field(description="검색할 소스 타입들")
        reasoning: str = Field(description="선택 이유")
    
    prompt = f"""
    사용자 질문: {state['query']}
    
    이 질문에 답하기 위해 검색해야 할 소스 타입을 선택하세요:
    - web: 일반적인 정보, 정의, 개념
    - database: 통계, 수치, 트렌드 데이터
    - document: 가이드, 베스트 프랙티스, 상세 설명
    - api: 기술 스펙, API 정보, 최신 업데이트
    
    여러 소스를 선택할 수 있습니다.
    """
    
    structured_llm = llm.with_structured_output(SourceDecision)
    decision = structured_llm.invoke(prompt)
    
    state["source_types"] = decision.source_types
    print(f"선택된 소스: {decision.source_types}")
    print(f"이유: {decision.reasoning}")
    
    return state

# 검색 노드
def retrieve_from_sources(state: MultiRAGState):
    """선택된 소스에서 정보 검색"""
    embeddings = OpenAIEmbeddings()
    all_docs = create_multi_source_data()
    
    # 벡터 저장소 생성
    vectorstore = FAISS.from_documents(all_docs, embeddings)
    
    # 소스별 필터링 검색
    retrieved = []
    for source_type in state.get("source_types", [SourceType.WEB]):
        # 모든 문서 검색 후 소스 필터링
        results = vectorstore.similarity_search(state["query"], k=10)
        filtered = [doc for doc in results if doc.metadata["source"] == source_type.value]
        retrieved.extend(filtered[:2])  # 각 소스에서 최대 2개
    
    state["retrieved_docs"] = retrieved
    state["sources_used"] = list(set([doc.metadata["source"] for doc in retrieved]))
    
    return state

# 답변 생성 노드
def generate_multi_source_answer(state: MultiRAGState):
    """검색된 정보로 답변 생성"""
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    
    # 컨텍스트 구성
    context = "\n\n".join([
        f"[{doc.metadata['source']}] {doc.page_content}"
        for doc in state["retrieved_docs"]
    ])
    
    prompt = f"""
    다음 정보를 바탕으로 질문에 답하세요:
    
    {context}
    
    질문: {state['query']}
    
    답변 시 사용한 소스를 명시하세요.
    """
    
    response = llm.invoke(prompt)
    state["answer"] = response.content
    
    return state

# Multi-RAG 시스템 생성
def create_multi_rag():
    workflow = StateGraph(MultiRAGState)
    
    # 노드 추가
    workflow.add_node("route", route_query)
    workflow.add_node("retrieve", retrieve_from_sources)
    workflow.add_node("answer", generate_multi_source_answer)
    
    # 엣지 추가
    workflow.set_entry_point("route")
    workflow.add_edge("route", "retrieve")
    workflow.add_edge("retrieve", "answer")
    workflow.add_edge("answer", END)
    
    return workflow.compile()

# 실행
multi_rag = create_multi_rag()

# 테스트 질문들
test_queries = [
    "Python이란 무엇인가요?",
    "Python 개발자 연봉은 얼마인가요?",
    "머신러닝 프로젝트를 시작하는 방법은?",
    "GPT-4의 스펙은?"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"질문: {query}\n")
    
    result = multi_rag.invoke({"query": query})
    
    print(f"\n사용된 소스: {', '.join(result['sources_used'])}")
    print(f"\n답변:\n{result['answer']}")


질문: Python이란 무엇인가요?

선택된 소스: [<SourceType.WEB: 'web'>, <SourceType.DOCUMENT: 'document'>]
이유: Python에 대한 일반적인 정보와 정의는 웹에서 쉽게 찾을 수 있으며, 가이드와 베스트 프랙티스는 문서에서 제공될 수 있습니다.

사용된 소스: web

답변:
Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다. 이는 다양한 프로그래밍 패러다임을 지원하며, 코드의 가독성이 높고, 배우기 쉬운 특성을 가지고 있습니다.

출처: [web] Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.

질문: Python 개발자 연봉은 얼마인가요?

선택된 소스: [<SourceType.WEB: 'web'>, <SourceType.DATABASE: 'database'>]
이유: 웹 소스는 Python 개발자 연봉에 대한 일반적인 정보와 개념을 제공할 수 있으며, 데이터베이스 소스는 연봉에 대한 통계와 트렌드 데이터를 제공할 수 있습니다.

사용된 소스: web

답변:
Python 개발자의 연봉은 지역, 경력, 기술 스택 등에 따라 다르기 때문에 정확한 금액을 제시하기는 어렵습니다. 일반적으로 미국에서는 Python 개발자의 평균 연봉이 약 80,000달러에서 120,000달러 사이로 보고되고 있습니다. 한국에서는 경력에 따라 4,000만 원에서 8,000만 원 이상일 수 있습니다.

이 정보는 특정 웹사이트에서 직접 확인한 것이 아니므로, 최신 연봉 정보를 얻으려면 구직 사이트나 연봉 조사 사이트를 참고하는 것이 좋습니다.

참고한 소스:
- [web] Python은 1991년 귀도 반 로섬이 개발한 고급 프로그래밍 언어입니다.
- [web] Django는 Python으로 작성된 오픈소스 웹 프레임워크입니다.

질문: 머신러닝 프로젝트를 시작하는 방법은?

선택된 소스: [<SourceType.WEB: 'web'>, <SourceType.DOCUMENT:

## 4. 고급 워크플로우 패턴

In [ ]:
from langgraph.graph import StateGraph, END
from typing import Literal

class WorkflowState(TypedDict):
    task: str
    complexity: Literal["simple", "medium", "complex"]
    steps: list
    current_step: int
    results: dict
    final_output: str

# 복잡도 평가
def assess_complexity(state: WorkflowState):
    """작업 복잡도 평가"""
    task = state["task"].lower()
    
    # 간단한 복잡도 평가 로직
    if any(word in task for word in ["간단", "기본", "확인"]):
        state["complexity"] = "simple"
    elif any(word in task for word in ["분석", "비교", "평가"]):
        state["complexity"] = "medium"
    else:
        state["complexity"] = "complex"
    
    print(f"작업 복잡도: {state['complexity']}")
    return state

# 작업 계획
def plan_steps(state: WorkflowState):
    """복잡도에 따른 작업 단계 계획"""
    
    if state["complexity"] == "simple":
        state["steps"] = ["직접 실행"]
    elif state["complexity"] == "medium":
        state["steps"] = ["데이터 수집", "분석", "결과 정리"]
    else:
        state["steps"] = ["요구사항 분석", "데이터 수집", "처리", "검증", "최종 정리"]
    
    state["current_step"] = 0
    state["results"] = {}
    
    print(f"계획된 단계: {state['steps']}")
    return state

# 단계 실행
def execute_step(state: WorkflowState):
    """현재 단계 실행"""
    if state["current_step"] >= len(state["steps"]):
        return state
    
    current = state["steps"][state["current_step"]]
    print(f"실행 중: {current}")
    
    # 단계별 실행 시뮬레이션
    state["results"][current] = f"{current} 완료"
    state["current_step"] += 1
    
    return state

# 완료 확인
def check_completion(state: WorkflowState) -> Literal["execute", "finalize"]:
    """모든 단계 완료 여부 확인"""
    if state["current_step"] < len(state["steps"]):
        return "execute"
    return "finalize"

# 최종 정리
def finalize_results(state: WorkflowState):
    """결과 최종 정리"""
    state["final_output"] = f"""
    작업: {state['task']}
    복잡도: {state['complexity']}
    완료된 단계: {', '.join(state['steps'])}
    결과: {state['results']}
    """
    return state

# 워크플로우 생성
def create_adaptive_workflow():
    workflow = StateGraph(WorkflowState)
    
    # 노드 추가
    workflow.add_node("assess", assess_complexity)
    workflow.add_node("plan", plan_steps)
    workflow.add_node("execute", execute_step)
    workflow.add_node("finalize", finalize_results)
    
    # 엣지 추가
    workflow.set_entry_point("assess")
    workflow.add_edge("assess", "plan")
    workflow.add_edge("plan", "execute")
    
    # 조건부 엣지
    workflow.add_conditional_edges(
        "execute",
        check_completion,
        {
            "execute": "execute",  # 재귀적 실행
            "finalize": "finalize"
        }
    )
    workflow.add_edge("finalize", END)
    
    return workflow.compile()

# 실행
adaptive_workflow = create_adaptive_workflow()

# 다양한 복잡도의 작업 테스트
tasks = [
    "간단한 데이터 확인",
    "사용자 행동 분석 및 리포트 생성",
    "머신러닝 모델 개발 및 배포 파이프라인 구축"
]

for task in tasks:
    print(f"\n{'='*60}")
    print(f"작업 시작: {task}\n")
    
    result = adaptive_workflow.invoke({"task": task})
    
    print(f"\n최종 결과:")
    print(result["final_output"])

## 실습 과제

1. 다국어 지원 챗봇 구현
2. 실시간 데이터 스트리밍을 처리하는 애플리케이션
3. 멀티모달 (텍스트+이미지) 처리 시스템 구축

In [ ]:
# 여기에 실습 코드를 작성하세요
